
<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesor: Víctor Navarro Aránguiz</em><br>

</div>

# Python para Data Science: Proyecto final
*15 de Julio de 2024*

**Fecha de Entrega**: 04 de Agosto de 2024

**Objetivos:**
  - Aplicar los conceptos aprendidos en clases.
  - Realizar el análisis de un dataset de su preferencia.
  - Familiarizarse con el manejo de versiones a través de Git.

**Puntaje Total:** 6 Puntos

**Nombre Estudiante(s)**: `Giuseppe Lavarello`

### Limpieza de Datos
##### Importe de Paquetes

In [ ]:
# Para manipulación de data 
import numpy as np
import pandas as pd

# Para visualización de data
from matplotlib import pyplot as plt
import seaborn as sns
import missingno as msno #de NaNs



##### Data Faltante

In [ ]:
# Como en el Análisis no cambiamos la data la podemos cargar de la misma manera


